In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/var/folders/z5/pd1h_8px20b3__mhlx7_s2yc0000gn/T/ipykernel_17746/3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [43]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

# Add these imports at the TOP of your code (or right before using lemmatize_text)
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk

# Download required NLTK data
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')  # Optional but recommended for better lemmatization

print("NLTK imports and downloads completed!")

NLTK imports and downloads completed!


[nltk_data] Downloading package wordnet to /Users/mac/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/mac/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/mac/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [6]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("/Users/mac/IronHacks/W7/D1/lab-natural-language-processing/data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


In [ ]:
print(data.head())
print("\nFirst Message:")
print(data['text'].iloc[0][:500])  # Mostra os primeiros 500 caracteres

                                                text  label
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...      1
1                                           Will do.      0
2  Nora--Cheryl has emailed dozens of memos about...      0
3  Dear Sir=2FMadam=2C I know that this proposal ...      1
4                                                fyi      0

Primeira mensagem:
DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL I AM MIKE CHUKWU , THE MANAGER, BILLS AND EXCHANGE AT THE FOREIGN REMITTANCE DEPARTMENT OF THE ZENITH INTERNATIONAL BANK PLC. I AM WRITING THIS LETTER TO ASK FOR YOUR SUPPORT AND COOPERATION TO CARRY OUT THIS BUSINESS OPPORTUNITY IN MY DEPARTMENT. WE DISCOVERED AN ABANDONED SUM OF $15,000,000.00 (FIFTEEN MILLION UNITED STATES DOLLARS ONLY) IN AN ACCOUNT THAT BELONGS TO ONE OF OUR FOREIGN CUSTOMERS WHO DIED ALONG WITH HIS ENTIRE FAMILY OF A WIFE AND 


### Let's divide the training and test set into two partitions

In [7]:
# Your code

train_data = pd.read_csv("/Users/mac/IronHacks/W7/D1/lab-natural-language-processing/data/kg_train.csv")
test_data = pd.read_csv("/Users/mac/IronHacks/W7/D1/lab-natural-language-processing/data/kg_test.csv")

## Data Preprocessing

In [8]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [23]:
# Your code

# First we remove inline JavaScript/CSS

import re

def remove_JS_CSS(text):
# Remove inline CSS styles: finds any <style> tag and all content until closing </style> tag

    text = re.sub(r'<style.*?>.*?</style>',             # Regex pattern to match <style> tags and their content
                    '',                                 # Replace matched content with empty string (remove it)
                    text,                               # The input text to process
                    flags=re.DOTALL | re.IGNORECASE     # Flags: DOTALL makes . match newlines, IGNORECASE handles case variations
                    )
        
    # Remove <style>...</style> tags and their content (duplicate operation - same as above but on variable 'texto')
    
    texto = re.sub(r'<style.*?>.*?</style>',            # Same regex pattern to match style tags
                    '',                                 # Replace with empty string to remove
                    texto,                              # Different variable name, same operation
                    flags=re.DOTALL | re.IGNORECASE     # Same flags for multiline and case-insensitive matching
    )
        
    return text


In [24]:
# Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters

def remove_coments_html(text):

    # Remove <!-- ... --> 
    text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)
    
    return text



In [25]:
# Next we can remove the remaining tags

def remover_tags_html(text):
 
    # Remove any HTML tag
    text = re.sub(r'<[^>]+>', '', text)
    
    return text

- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [29]:
# Your code

def clean_text_complete(text):
    
    # 1. Convert to lowercase
    text = text.lower()
    
    # 2. Remove prefixed 'b' (common in encoding)
    text = re.sub(
        r'^b[\'\"]',    # Pattern: 'b' at start + quote
        '',             # Replace with empty string
        text            # Input text
    )
    
    # 3. Remove special characters
    text = re.sub(
        r'[^a-zA-Z0-9\s]',  # Pattern: keep only alphanumeric and spaces
        '',                 # Replace with empty string
        text                # Input text
    )
    
    # 4. Remove numbers
    text = re.sub(
        r'\d+',         # Pattern: one or more digits
        '',             # Replace with empty string
        text            # Input text
    )
    
    # 5. Remove all single characters
    text = re.sub(
        r'\s+[a-zA-Z]\s+',  # Pattern: space + single letter + space
        ' ',                # Replace with single space
        text                # Input text
    )
    
    # 6. Remove single characters from the start
    text = re.sub(
        r'^[a-zA-Z]\s+',    # Pattern: single letter at start + spaces
        '',                 # Replace with empty string
        text                # Input text
    )
    
    # 7. Substitute multiple spaces with single space
    text = re.sub(
        r'\s+',         # Pattern: one or more whitespace characters
        ' ',            # Replace with single space
        text            # Input text
    )
    
    return text.strip()  # Final cleanup - remove extra spaces

## Now let's work on removing stopwords
Remove the stopwords.

In [33]:
# Your code

def remove_stopwords(text):
    
    # Get english stopwords list
    stop_words = set(stopwords.words('english'))            #Load English stopwords
    
    # Split text ino words
    words = text.split()                                    #convert text to list of words
    
    # Create empty list for filtered words
    
    filtered_words = []
    
    # Loop through each word and append if not stopword
    for word in words:
        if word not in stop_words:
            filtered_words.append(word)
    
    # Join words vacl into text
    cleaned_text = ' '.join(filtered_words)
    
    return cleaned_text

## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [35]:
# Your code

def lemmatize_text(text):
    
    # Initialize lemmatizer
    lemmatizer = WordNetLemmatizer()        #Create Lemmatizer Object
    
    # Tokenize text into words
    words = word_tokenize(text)              #Input text to tokenize
    
    # Lemmatize each word
    lemmatized_words = [
        lemmatizer.lemmatize(
            word  # Current word to lemmatize
        ) 
        for word in words  # For each word in the list
    ]
    
    # Join words back into text
    lemmatized_text = ' '.join(
        lemmatized_words  # Join lemmatized words with spaces
    )
    
    return lemmatized_text

## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [36]:
# Your code

from collections import Counter
import pandas as pd

def get_top_words_by_label(dataframe, text_column, label_column, top_n=10):
    """
    Get top N words for each label in the dataset
    """
    # Separate data by label
    ham_messages = dataframe[
        dataframe[label_column] == 0  # Filter rows where label is 0 (ham)
    ]
    
    spam_messages = dataframe[
        dataframe[label_column] == 1  # Filter rows where label is 1 (spam)
    ]
    
    # Extract all words from ham messages
    ham_words = []
    for message in ham_messages[text_column]:  # For each ham message
        words = message.split()                # Split into words
        ham_words.extend(words)                # Add to ham words list
    
    # Extract all words from spam messages  
    spam_words = []
    for message in spam_messages[text_column]:  # For each spam message
        words = message.split()                 # Split into words
        spam_words.extend(words)                # Add to spam words list
    
    # Count word frequencies for ham
    ham_word_counts = Counter(
        ham_words  # Count occurrences of each word
    )
    
    # Count word frequencies for spam
    spam_word_counts = Counter(
        spam_words  # Count occurrences of each word
    )
    
    # Get top N words for each category
    top_ham_words = ham_word_counts.most_common(
        top_n  # Get top N most common words
    )
    
    top_spam_words = spam_word_counts.most_common(
        top_n  # Get top N most common words
    )
    
    return top_ham_words, top_spam_words

# 🧪 GET TOP 10 WORDS FOR HAM AND SPAM
top_ham, top_spam = get_top_words_by_label(
    dataframe=data,           # Your dataframe
    text_column='text',       # Column with text messages
    label_column='label',     # Column with labels (0=ham, 1=spam)
    top_n=10                 # Get top 10 words
)

# 📈 DISPLAY RESULTS
print("=== TOP 10 WORDS IN HAM MESSAGES ===")
for word, count in top_ham:
    print(f"{word}: {count} times")

print("\n=== TOP 10 WORDS IN SPAM MESSAGES ===")
for word, count in top_spam:
    print(f"{word}: {count} times")

=== TOP 10 WORDS IN HAM MESSAGES ===
the: 1593 times
to: 1039 times
of: 782 times
and: 781 times
a: 591 times
in: 540 times
that: 371 times
is: 353 times
for: 344 times
on: 283 times

=== TOP 10 WORDS IN SPAM MESSAGES ===
the: 5676 times
to: 4688 times
of: 4118 times
and: 3234 times
in: 2672 times
I: 2633 times
you: 2273 times
this: 2010 times
a: 1939 times
for: 1685 times


In [44]:
# Apply your existing functions in sequence
data_train['preprocessed_text'] = data_train['text'].apply(clean_text_complete)
data_train['preprocessed_text'] = data_train['preprocessed_text'].apply(remove_stopwords)
data_train['preprocessed_text'] = data_train['preprocessed_text'].apply(lemmatize_text)

data_val['preprocessed_text'] = data_val['text'].apply(clean_text_complete)
data_val['preprocessed_text'] = data_val['preprocessed_text'].apply(remove_stopwords)
data_val['preprocessed_text'] = data_val['preprocessed_text'].apply(lemmatize_text)

print("preprocessed_text column created!")
print("Sample:", data_train['preprocessed_text'].iloc[0])

preprocessed_text column created!
Sample: dear sir strictly private business proposal mike chukwu manager bill exchange foreign remittance department zenith international bank plc writing letter ask support cooperation carry business opportunity department discovered abandoned sum fifteen million united state dollar account belongs one foreign customer died along entire family wife two child november plane crash since heard death expecting nextofkin come put claim money heirbecause can not release fund account unless someone applies claim nextofkin deceased indicated banking guideline unfortunately neither family member distant relative ever appeared claim said fund upon discoveryi official department agreed make business release total amount account heir fund since one came discovered maintained account bank otherwise fund returned bank treasury unclaimed fund agreed ratio sharing stated thus foreign partner u official department settlement local foreign expences incurred u course bus

## Extra features

In [45]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).

data_train = train_data
data_val = test_data

money_simbol_list = "|".join(["euro","dollar","pound","€",r"\$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data_train.head()

,text,label,preprocessed_text,money_mark,suspicious_words,text_len
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1,dear sir strictly private business proposal mi...,1,1,1505
1,Will do.,0,,0,0,0
2,Nora--Cheryl has emailed dozens of memos about...,0,noracheryl emailed dozen memo haiti weekend pl...,0,0,110
3,Dear Sir=2FMadam=2C I know that this proposal ...,1,dear sirfmadamc know proposal might surprise e...,1,1,1382
4,fyi,0,fyi,0,0,3


## How would work the Bag of Words with Count Vectorizer concept?

In [ ]:
# Your code

## TF-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [46]:
# Your code

from sklearn.feature_extraction.text import CountVectorizer

def bag_of_words_with_count_vectorizer(texts):
    """
    Implement Bag of Words using CountVectorizer
    """
    # Initialize CountVectorizer
    vectorizer = CountVectorizer(
        max_features=1000,           # Keep only top 1000 most frequent words
        stop_words='english',        # Remove English stopwords
        ngram_range=(1, 2)          # Use single words and word pairs
    )
    
    # Fit and transform the texts
    X = vectorizer.fit_transform(
        texts  # Input texts to vectorize
    )
    
    # Get feature names (vocabulary)
    feature_names = vectorizer.get_feature_names_out()
    
    return X, feature_names, vectorizer

# 🧪 APPLY TO OUR PREPROCESSED DATA
print("Applying Bag of Words with CountVectorizer...")

# Use preprocessed_text for better results
X_train_bow, feature_names, bow_vectorizer = bag_of_words_with_count_vectorizer(
    data_train['preprocessed_text']  # Use cleaned text
)

X_val_bow = bow_vectorizer.transform(
    data_val['preprocessed_text']    # Use same vocabulary for validation
)

# 📊 DISPLAY RESULTS
print("=== BAG OF WORDS RESULTS ===")
print(f"Training data shape: {X_train_bow.shape}")
print(f"Validation data shape: {X_val_bow.shape}")
print(f"Number of features (vocabulary size): {len(feature_names)}")
print(f"First 20 features: {feature_names[:20]}")

# Show a sample of the vectorized data
print(f"\n=== SAMPLE VECTORIZED DATA ===")
print("Original text sample:")
print(data_train['preprocessed_text'].iloc[0][:100] + "...")

print("\nVectorized representation (first 10 features):")
sample_vector = X_train_bow[0].toarray()[0][:10]
for i, value in enumerate(sample_vector):
    if value > 0:
        print(f"  {feature_names[i]}: {value}")

# 🎯 CREATE DATAFRAME FOR BETTER VISUALIZATION (optional)
bow_df = pd.DataFrame(
    X_train_bow.toarray(),          # Convert sparse matrix to dense
    columns=feature_names           # Use feature names as columns
)

print(f"\n=== BAG OF WORDS DATAFRAME (first 5 rows, first 10 features) ===")
print(bow_df.iloc[:5, :10])

Applying Bag of Words with CountVectorizer...
=== BAG OF WORDS RESULTS ===
Training data shape: (5964, 1000)
Validation data shape: (5964, 1000)
Number of features (vocabulary size): 1000
First 20 features: ['abacha' 'abandoned' 'abandoned sum' 'abidjan' 'able' 'abroad' 'ac'
 'accept' 'acceptance' 'access' 'according' 'according percentage'
 'account' 'act' 'action' 'actual' 'actually' 'address' 'administration'
 'advice']

=== SAMPLE VECTORIZED DATA ===
Original text sample:
dear sir strictly private business proposal mike chukwu manager bill exchange foreign remittance dep...

Vectorized representation (first 10 features):
  abandoned: 1
  abandoned sum: 1

=== BAG OF WORDS DATAFRAME (first 5 rows, first 10 features) ===
   abacha  abandoned  abandoned sum  abidjan  able  abroad  ac  accept  \
0       0          1              1        0     0       0   0       0   
1       0          0              0        0     0       0   0       0   
2       0          0              0        0 

## And the Train a Classifier?

In [49]:
# Yor code

from sklearn.model_selection import train_test_split

# Since data_val doesn't have labels, let's split data_train for training and validation
print("Splitting data_train into training and validation sets...")

# Split data_train into train and validation (80% train, 20% validation)
X_temp = data_train.drop('label', axis=1)  # Features
y_temp = data_train['label']               # Labels

X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_temp, 
    y_temp, 
    test_size=0.2, 
    random_state=42,
    stratify=y_temp  # Keep same distribution of spam/ham
)

print(f"Training set: {len(X_train_split)} samples")
print(f"Validation set: {len(X_val_split)} samples")
print(f"Spam ratio in training: {y_train_split.mean():.3f}")
print(f"Spam ratio in validation: {y_val_split.mean():.3f}")

# 🎯 NOW APPLY BAG OF WORDS TO THE SPLIT DATA
print("\nApplying Bag of Words to split data...")

# Fit CountVectorizer on training split and transform both
X_train_bow_split, feature_names, bow_vectorizer = bag_of_words_with_count_vectorizer(
    X_train_split['preprocessed_text']  # Training texts
)

X_val_bow_split = bow_vectorizer.transform(
    X_val_split['preprocessed_text']    # Validation texts
)

print(f"Training BOW shape: {X_train_bow_split.shape}")
print(f"Validation BOW shape: {X_val_bow_split.shape}")

# 🚀 TRAIN CLASSIFIER WITH PROPER VALIDATION
print("\nTraining classifier with proper validation set...")

classifier_bow, predictions_bow, accuracy_bow = train_and_evaluate_classifier(
    X_train=X_train_bow_split,      # Training features from split
    X_val=X_val_bow_split,          # Validation features from split
    y_train=y_train_split,          # Training labels from split
    y_val=y_val_split,              # Validation labels from split
    classifier_name="Bag of Words Classifier"
)

# 💡 EXTRA: COMBINE WITH YOUR EXTRA FEATURES
print("\n=== COMBINING BOW WITH EXTRA FEATURES ===")

# Get the extra features from the split data
extra_features_train = X_train_split[['money_mark', 'suspicious_words', 'text_len']].values
extra_features_val = X_val_split[['money_mark', 'suspicious_words', 'text_len']].values

# Combine BOW with extra features
from scipy.sparse import hstack

X_train_combined = hstack([X_train_bow_split, extra_features_train])
X_val_combined = hstack([X_val_bow_split, extra_features_val])

print(f"Combined features shape - Train: {X_train_combined.shape}")
print(f"Combined features shape - Val: {X_val_combined.shape}")

# Train classifier with combined features
print("\nTraining classifier with BOW + Extra Features...")
classifier_combined, predictions_combined, accuracy_combined = train_and_evaluate_classifier(
    X_train=X_train_combined,      # Combined features
    X_val=X_val_combined,          # Combined features
    y_train=y_train_split,         # Training labels
    y_val=y_val_split,             # Validation labels
    classifier_name="BOW + Extra Features Classifier"
)

# 📊 COMPARE PERFORMANCE
print("\n=== PERFORMANCE COMPARISON ===")
print(f"Bag of Words Only: {accuracy_bow:.4f} ({accuracy_bow*100:.2f}%)")
print(f"BOW + Extra Features: {accuracy_combined:.4f} ({accuracy_combined*100:.2f}%)")

Splitting data_train into training and validation sets...
Training set: 4771 samples
Validation set: 1193 samples
Spam ratio in training: 0.432
Spam ratio in validation: 0.433

Applying Bag of Words to split data...
Training BOW shape: (4771, 1000)
Validation BOW shape: (1193, 1000)

Training classifier with proper validation set...
Training Bag of Words Classifier...
=== BAG OF WORDS CLASSIFIER RESULTS ===
Accuracy: 0.9472 (94.72%)

=== CLASSIFICATION REPORT ===
              precision    recall  f1-score   support

         Ham       0.94      0.96      0.95       677
        Spam       0.95      0.92      0.94       516

    accuracy                           0.95      1193
   macro avg       0.95      0.94      0.95      1193
weighted avg       0.95      0.95      0.95      1193


=== CONFUSION MATRIX ===
[[653  24]
 [ 39 477]]

=== COMBINING BOW WITH EXTRA FEATURES ===
Combined features shape - Train: (4771, 1003)
Combined features shape - Val: (1193, 1003)

Training classifier wi

### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to **find the most relevant features**.

For example, you can test the following options and check which of them performs better:
- Using "Bag of Words" only
- Using "TF-IDF" only
- Bag of Words + extra flags (money_mark, suspicious_words, text_len)
- TF-IDF + extra flags


You can work with teams of two persons (recommended).

In [50]:
# Your code

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import numpy as np

# Initialize the classifier (FIXED - cannot be changed)
classifier = MultinomialNB()

def evaluate_feature_set(X_train, X_val, y_train, y_val, feature_name):
    """
    Evaluate a specific feature set with the fixed classifier
    """
    # Train the classifier
    classifier.fit(X_train, y_train)
    
    # Make predictions
    y_pred = classifier.predict(X_val)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_val, y_pred)
    
    print(f"{feature_name}: {accuracy:.4f} ({accuracy*100:.2f}%)")
    return accuracy

# 🎯 FEATURE SET 1: BAG OF WORDS ONLY
print("=== BAG OF WORDS ONLY ===")
X_train_bow, bow_features, bow_vectorizer = bag_of_words_with_count_vectorizer(X_train_split['preprocessed_text'])
X_val_bow = bow_vectorizer.transform(X_val_split['preprocessed_text'])
accuracy_bow = evaluate_feature_set(X_train_bow, X_val_bow, y_train_split, y_val_split, "Bag of Words")

# 🎯 FEATURE SET 2: TF-IDF ONLY
print("\n=== TF-IDF ONLY ===")
def create_tfidf_features(texts_train, texts_val):
    """Create TF-IDF features"""
    tfidf_vectorizer = TfidfVectorizer(
        max_features=1000,
        stop_words='english',
        ngram_range=(1, 2)
    )
    X_train_tfidf = tfidf_vectorizer.fit_transform(texts_train)
    X_val_tfidf = tfidf_vectorizer.transform(texts_val)
    return X_train_tfidf, X_val_tfidf, tfidf_vectorizer

X_train_tfidf, X_val_tfidf, tfidf_vectorizer = create_tfidf_features(
    X_train_split['preprocessed_text'], 
    X_val_split['preprocessed_text']
)
accuracy_tfidf = evaluate_feature_set(X_train_tfidf, X_val_tfidf, y_train_split, y_val_split, "TF-IDF")

# 🎯 FEATURE SET 3: BAG OF WORDS + EXTRA FEATURES
print("\n=== BAG OF WORDS + EXTRA FEATURES ===")
from scipy.sparse import hstack

# Get extra features
extra_features_train = X_train_split[['money_mark', 'suspicious_words', 'text_len']].values
extra_features_val = X_val_split[['money_mark', 'suspicious_words', 'text_len']].values

# Combine BOW with extra features
X_train_bow_extra = hstack([X_train_bow, extra_features_train])
X_val_bow_extra = hstack([X_val_bow, extra_features_val])
accuracy_bow_extra = evaluate_feature_set(X_train_bow_extra, X_val_bow_extra, y_train_split, y_val_split, "BOW + Extra Features")

# 🎯 FEATURE SET 4: TF-IDF + EXTRA FEATURES
print("\n=== TF-IDF + EXTRA FEATURES ===")
X_train_tfidf_extra = hstack([X_train_tfidf, extra_features_train])
X_val_tfidf_extra = hstack([X_val_tfidf, extra_features_val])
accuracy_tfidf_extra = evaluate_feature_set(X_train_tfidf_extra, X_val_tfidf_extra, y_train_split, y_val_split, "TF-IDF + Extra Features")

# 📊 COMPARISON SUMMARY
print("\n" + "="*50)
print("FEATURE SET COMPARISON SUMMARY")
print("="*50)

results = {
    "Bag of Words": accuracy_bow,
    "TF-IDF": accuracy_tfidf,
    "BOW + Extra Features": accuracy_bow_extra,
    "TF-IDF + Extra Features": accuracy_tfidf_extra
}

# Sort by performance
sorted_results = sorted(results.items(), key=lambda x: x[1], reverse=True)

print("\nRANKED BY PERFORMANCE:")
for i, (name, acc) in enumerate(sorted_results, 1):
    print(f"{i}. {name}: {acc:.4f} ({acc*100:.2f}%)")

# 🏆 TRAIN FINAL MODEL WITH BEST FEATURES
print("\n" + "="*50)
print("TRAINING FINAL MODEL WITH BEST FEATURES")
print("="*50)

best_feature_name = sorted_results[0][0]
print(f"Best performing feature set: {best_feature_name}")

if best_feature_name == "Bag of Words":
    X_final_train = X_train_bow
    X_final_val = X_val_bow
    final_vectorizer = bow_vectorizer
elif best_feature_name == "TF-IDF":
    X_final_train = X_train_tfidf
    X_final_val = X_val_tfidf
    final_vectorizer = tfidf_vectorizer
elif best_feature_name == "BOW + Extra Features":
    X_final_train = X_train_bow_extra
    X_final_val = X_val_bow_extra
    final_vectorizer = bow_vectorizer
else:  # TF-IDF + Extra Features
    X_final_train = X_train_tfidf_extra
    X_final_val = X_val_tfidf_extra
    final_vectorizer = tfidf_vectorizer

# Train final model on entire training split
final_classifier = MultinomialNB()
final_classifier.fit(X_final_train, y_train_split)

# Final validation performance
final_predictions = final_classifier.predict(X_final_val)
final_accuracy = accuracy_score(y_val_split, final_predictions)

print(f"Final Model Accuracy: {final_accuracy:.4f} ({final_accuracy*100:.2f}%)")

# 🎯 PREPARE KAGGLE SUBMISSION
print("\n" + "="*50)
print("PREPARING KAGGLE SUBMISSION")
print("="*50)

# Prepare test data (data_val) using the same preprocessing
if best_feature_name in ["Bag of Words", "BOW + Extra Features"]:
    X_test = bow_vectorizer.transform(data_val['preprocessed_text'])
else:
    X_test = tfidf_vectorizer.transform(data_val['preprocessed_text'])

# Add extra features if needed
if best_feature_name in ["BOW + Extra Features", "TF-IDF + Extra Features"]:
    test_extra_features = data_val[['money_mark', 'suspicious_words', 'text_len']].values
    X_test = hstack([X_test, test_extra_features])

# Make predictions on test set
test_predictions = final_classifier.predict(X_test)

# Create submission file
submission = pd.DataFrame({
    'text': data_val['text'],
    'label': test_predictions
})

print(f"Test predictions ready: {len(test_predictions)} samples")
print(f"Spam predictions: {test_predictions.sum()} ({test_predictions.mean()*100:.2f}%)")
print(f"Ham predictions: {len(test_predictions) - test_predictions.sum()}")

# Save submission file
submission_file = 'kaggle_submission.csv'
submission[['text', 'label']].to_csv(submission_file, index=False)
print(f"Submission file saved: {submission_file}")

# 📈 FEATURE IMPORTANCE ANALYSIS
print("\n" + "="*50)
print("FEATURE IMPORTANCE ANALYSIS")
print("="*50)

if best_feature_name in ["Bag of Words", "BOW + Extra Features"]:
    feature_names = bow_vectorizer.get_feature_names_out()
else:
    feature_names = tfidf_vectorizer.get_feature_names_out()

# Add extra feature names if used
if best_feature_name in ["BOW + Extra Features", "TF-IDF + Extra Features"]:
    feature_names = np.append(feature_names, ['money_mark', 'suspicious_words', 'text_len'])

# Get feature importance (log probabilities difference)
spam_probs = final_classifier.feature_log_prob_[1]
ham_probs = final_classifier.feature_log_prob_[0]
importance_scores = spam_probs - ham_probs

print("Top 15 features most predictive of SPAM:")
top_spam_indices = importance_scores.argsort()[-15:][::-1]
for idx in top_spam_indices:
    if idx < len(feature_names):
        print(f"  {feature_names[idx]}: {importance_scores[idx]:.4f}")

print("\nTop 15 features most predictive of HAM:")
top_ham_indices = importance_scores.argsort()[:15]
for idx in top_ham_indices:
    if idx < len(feature_names):
        print(f"  {feature_names[idx]}: {importance_scores[idx]:.4f}")

=== BAG OF WORDS ONLY ===
Bag of Words: 0.9472 (94.72%)

=== TF-IDF ONLY ===
TF-IDF: 0.9581 (95.81%)

=== BAG OF WORDS + EXTRA FEATURES ===
BOW + Extra Features: 0.9489 (94.89%)

=== TF-IDF + EXTRA FEATURES ===
TF-IDF + Extra Features: 0.9581 (95.81%)

FEATURE SET COMPARISON SUMMARY

RANKED BY PERFORMANCE:
1. TF-IDF: 0.9581 (95.81%)
2. TF-IDF + Extra Features: 0.9581 (95.81%)
3. BOW + Extra Features: 0.9489 (94.89%)
4. Bag of Words: 0.9472 (94.72%)

TRAINING FINAL MODEL WITH BEST FEATURES
Best performing feature set: TF-IDF
Final Model Accuracy: 0.9581 (95.81%)

PREPARING KAGGLE SUBMISSION
Test predictions ready: 5964 samples
Spam predictions: 2493 (41.80%)
Ham predictions: 3471
Submission file saved: kaggle_submission.csv

FEATURE IMPORTANCE ANALYSIS
Top 15 features most predictive of SPAM:
  transaction: 3.5839
  bank: 3.5266
  account: 3.5165
  fund: 3.5072
  kin: 3.4408
  money: 3.4003
  sum: 3.3340
  transfer: 3.3007
  foreigner: 2.8506
  nigeria: 2.7123
  dollar: 2.7012
  deposit